# Example 132: Undulator Flux Multi-Particle

In this example multi-particle flux calculation from an undulator for a non-zero emittance electron beam.

In [ ]:
# This has nothing to do with OSCARS, but it puts the matplotlib plots inline in the notebook
%matplotlib inline

# Import the OSCARS SR module
import oscars.sr

# Import basic plot utilities (matplotlib).  You don't need these to run OSCARS, but it's used here for basic plots
from oscars.plots_mpl import *

In [ ]:
# Create a new OSCARS object.  Default to 8 threads and always use the GPU if available
osr = oscars.sr.sr(nthreads=8, gpu=1)

## Create the undulator field

Here we create the undulator field.  Here *bfield* represents maximum magnetic field [$B_x, B_y, B_z$].  The *period* is also in vector form which allows you to orient the axis of the undulator in any arbitrary direction.  The number of periods is given by *nperiods*.  This is the number of FULL periods.  A terminating field of 1 period length is added to each side in addition to *nperiods*.  Typically clear_magnetic_fields() is called before adding a field in notebooks only to save time when making changes and rerunning sections of the code so it is not strictly necessary.

In [ ]:
# Clear any existing fields (just good habit in notebook style) and add an undulator field
osr.clear_bfields()
osr.add_bfield_undulator(bfield=[0, 1, 0], period=[0, 0, 0.049], nperiods=31)

# Just to check the field that we added seems visually correct
plot_bfield(osr)

## Add a particle beam

Here we add a particle beam making use of some of the defaults, namely:
    * type='electron'
    * t0=0
    * d0=[0, 0, 1]

One must specify ctstartstop.  This is the start and stop time of the calculation.  In this example we will start the calculation at t=0 and go to t=2 (given in units of ct) since the beam is relativistic.  In this example you can specify the start time as less than 0 which is useful if you want to propogate the particle backwars in time.  This is useful for instance if you have a bending magnet before the undulator that you wish to include.

In [ ]:
# Setup beam similar to NSLSII
osr.set_particle_beam(
    energy_GeV=3,
    x0=[0, 0, -1],
    current=0.500,
    sigma_energy_GeV=0.001*3,
    beta=[1.5, 0.8],
    emittance=[0.9e-9, 0.008e-9]
)

# Set the start and stop times for the calculation
osr.set_ctstartstop(0, 2)

## Calculate Trajectory

Now we calculate the trajectory and plot it.  It is enough to call calculate_trajectory().  If you are doing other calculations (flux, spectra, power density) it is not necesary to call this since it is called internally.

In [ ]:
# Run the particle trajectory calculation on the ideal case
osr.set_new_particle(particle='ideal')
trajectory = osr.calculate_trajectory()

# Plot the trajectory position and velocity
plot_trajectory_position(trajectory)

## Calculate Spectrum

Calculate the spectrum in a given range with the given number of points equally spaced (not a requirement).

First we calculate the single eletron spectrum for the idea particle, then we calculate the multi-particle spectrum.  One can use nthreads or gpu in any of these calculations.  For multi-particle calculations simply add *nparticles=100* and so on depending on the number of particles you wish to simulate.

In [ ]:
# Single particle spectrum for the ideal case
osr.set_new_particle(particle='ideal')
spectrum_se = osr.calculate_spectrum(obs=[0, 0, 30], energy_range_eV=[130, 160], npoints=500)
spectrum_me = osr.calculate_spectrum(obs=[0, 0, 30], energy_range_eV=[130, 160], npoints=500, nparticles=100)
plot_spectra([spectrum_se, spectrum_me], ['single-electron', 'multi-electron'])

## Calculate Flux

Calcuate the flux on a rectangular surface downstream 30 [m] from the center of the device.

In [ ]:
# Calcuate flux at 30 [m] for 152 eV
flux_multi = osr.calculate_flux_rectangle(
    plane='XY',
    energy_eV=152,
    width=[0.01, 0.01],
    npoints=[101, 101],
    translation=[0, 0, 30],
    nparticles=5
)

plot_flux(flux_multi, ofile='UndulatorFlux_MultiParticle.png')